# **因子分析**

因子分析是一种多变量统计技术，旨在通过识别潜在的、未被直接观测到的因子来解释观测变量之间的相关性。因子分析分为两种主要类型：探索性因子分析（Exploratory Factor Analysis, EFA）和确认性因子分析（Confirmatory Factor Analysis, CFA）。



In [71]:
import pandas as pd
import numpy as np

df = pd.read_excel("data\\evaluation.xlsx").drop("序号", axis=1)
df.head()

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10
0,1.5,7.1,280,424,4,430,90,112,459,453
1,1.5,7.1,280,424,4,431,92,112,459,453
2,1.5,8.7,280,424,4,431,92,112,459,453
3,1.5,8.7,280,424,4,431,92,112,459,448
4,1.5,8.7,280,424,4,431,90,112,455,453


## ***探索性因子分析（EFA）***

探索性因子分析的目的是从数据中探索潜在的因子结构，而不是验证预先设定的模型。在没有明确假设的情况下，EFA用于确定哪些变量聚集在一起形成因子。

<br><br>

KMO 检验用于测量样本充足性，它的值在0到1之间。KMO 值越高，数据越适合进行因子分析。常见的解释标准如下：

- KMO > 0.9：非常好
- 0.8 < KMO ≤ 0.9：好
- 0.7 < KMO ≤ 0.8：中等
- 0.6 < KMO ≤ 0.7：较差
- KMO < 0.6：不适合

p<0.05：拒绝原假设（相关矩阵不是单位矩阵），适合进行因子分析

<br><br>

In [72]:
from factor_analyzer.factor_analyzer import calculate_kmo, calculate_bartlett_sphericity

# KMO 检验
kmo_all, kmo_model = calculate_kmo(df)

# Bartlett's 检验
chi_square_value, p_value = calculate_bartlett_sphericity(df)

print("KMO Model:", kmo_model)
print("Bartlett's Test——Chi-Square:", chi_square_value)
print("Bartlett's Test——P-value:", p_value)

KMO Model: 0.6933381738019926
Bartlett's Test——Chi-Square: 78.4746315028124
Bartlett's Test——P-value: 0.0014717479314208276


In [73]:
from factor_analyzer import FactorAnalyzer

# 因子分析
fa = FactorAnalyzer(n_factors=4, rotation='varimax')
fa.fit(df)

# 输出因子载荷矩阵
loadings = fa.loadings_
print("Factor Loadings:")
print(loadings)

Factor Loadings:
[[-0.30110085  0.03862989  0.95132559 -0.0519133 ]
 [-0.84313456  0.21847548  0.22137182  0.05597841]
 [ 0.73891447  0.0956507  -0.03364268 -0.21093229]
 [-0.8222621   0.31509171  0.29248251 -0.01944084]
 [ 0.52280712 -0.74395308 -0.09777055  0.13572706]
 [ 0.08627904  0.73918789 -0.01213552  0.04848961]
 [-0.19799138 -0.09670839 -0.05832723  0.97364517]
 [-0.17961429  0.44062153  0.23025079 -0.06178171]
 [ 0.38467098 -0.56518273  0.13956737  0.21412175]
 [ 0.7095415  -0.30391161 -0.20541436 -0.07897535]]


In [74]:
# 因子解释方差
variance = fa.get_factor_variance()
variance_df = pd.DataFrame(variance, 
                           columns=['Factor1', 'Factor2', 'Factor3', 'Factor4'], 
                           index=['SS Loadings', 'Proportion Var', 'Cumulative Var']
                           )
variance_df

,Factor1,Factor2,Factor3,Factor4
SS Loadings,3.027301,1.872814,1.168502,1.075359
Proportion Var,0.302730,0.187281,0.116850,0.107536
Cumulative Var,0.302730,0.490012,0.606862,0.714398


## ***确认性因子分析（CFA）***
确认性因子分析用于验证预先设定的因子模型。CFA 假设特定的因子结构，并检验数据是否支持这种假设。

In [75]:
from semopy import Model

# 定义模型
model_desc = """
F1 =~ X_1 + X_2
F2 =~ X_3 + X_4
F3 =~ X_5 + X_6
F4 =~ X_7 + X_8
"""

# 构建和拟合模型
model = Model(model_desc)
model.fit(df)


SolverResult(fun=0.8216284292120868, success=True, n_it=97, x=array([ 4.68676791e+00, -4.80031999e-01, -2.22225352e-01, -6.53885506e+02,
        3.61459099e-02, -1.83001412e-01, -1.25137020e-01, -9.64756108e-03,
        9.64329363e-01,  6.94329215e-01,  4.49782103e-02,  9.99918427e-01,
        6.79928672e-02,  1.11675635e-02,  7.03129574e-02,  3.83332466e-01,
        1.50469213e+00,  8.89935166e-18,  0.00000000e+00,  1.23455825e-01,
        7.91264618e-01,  4.94134632e+03]), message='Optimization terminated successfully', name_method='SLSQP', name_obj='MLW')

因子载荷的显著性：

当因子载荷的 p 值显著时，说明该观察变量与潜在因子的关系在数据中是重要的。比如，如果 X_2 对 F1 的载荷显著，这表明 X_2 很好地反映了 F1。
因子间相关的显著性：

如果因子间的相关性 p 值显著，这说明因子之间存在统计显著的关系。这有助于验证理论模型中因子间的假设关系。

这个部分有三组数据，第一组是`变量~因子`，第二组是`因子~~因子`，第三组是`变量~~变量`

- 第一组是看这个变量到底是不是归在这个因子里，有没有问题？？？换言之，这个变量对这个因子的影响大不大，p<0.05就是大，就是设置的对
- 第二组是看因子之间有没有关系，比如F1和F2之间有没有关系，p<0.05就是有关系（我们不希望有关系！！！）
- 第三组是看变量之间有没有关系，比如X1和X2之间有没有关系，p<0.05就是有关系

In [76]:

# 查看模型结果
params = model.inspect()
params

,lval,op,rval,Estimate,Std. Err,z-value,p-value
0,X_1,~,F1,1.000000e+00,-,-,-
1,X_2,~,F1,4.686768e+00,1.75039,2.677557,0.007416
2,X_3,~,F2,1.000000e+00,-,-,-
3,X_4,~,F2,-4.800320e-01,0.160073,-2.998829,0.00271
4,X_5,~,F3,1.000000e+00,-,-,-
5,X_6,~,F3,-2.222254e-01,0.110015,-2.019952,0.043388
6,X_7,~,F4,1.000000e+00,-,-,-
7,X_8,~,F4,-6.538855e+02,1993.709614,-0.327974,0.742931
8,F1,~~,F1,3.614591e-02,0.028002,1.290825,0.196764
9,F1,~~,F2,-1.830014e-01,0.106152,-1.723957,0.084716
